In [209]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import joblib
import yfinance as yf
from datetime import datetime, timedelta
from scipy import stats

def get_stock_ratings(ticker_symbols):
    """
    Generate risk assessments and ratings for multiple ticker symbols.
    
    Args:
        ticker_symbols (list): List of ticker symbols (e.g., ["AAPL", "MSFT"])
    
    Returns:
        dict: Dictionary with ticker symbols as keys and their risk/rating assessments
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    seq_length = 30
    prediction_days = 60  # For forward-looking assessment
    
    # Load the trained model and scaler for future predictions
    try:
        model = BiLSTMModel().to(device)
        model.load_state_dict(torch.load('bilstm_model.pth', map_location=device))
        model.eval()
        scaler_diff = joblib.load('scaler.joblib')
    except Exception as e:
        print(f"Warning: {e}")
        # Continue without model predictions if loading fails
        model = None
        scaler_diff = None
    
    results = {}
    
    for ticker in ticker_symbols:
        try:
            # Get historical data
            ticker_obj = yf.Ticker(ticker)
            hist_data = ticker_obj.history(period="3y", interval="1d")  # 3 years of data for better analysis
            
            if hist_data.empty or len(hist_data) < 252:  # Need at least 1 year of data
                results[ticker] = {"error": f"Insufficient historical data for {ticker}"}
                continue
                
            # Extract close prices
            close_prices = hist_data['Close'].values.astype(float)
            
            # ------- RISK CALCULATION -------
            # Calculate daily returns
            daily_returns = np.diff(close_prices) / close_prices[:-1]
            
            # Calculate annualized volatility (standard deviation of returns * sqrt(252))
            volatility_annual = np.std(daily_returns) * np.sqrt(252) * 100  # as percentage
            
            # Determine risk level based on volatility
            if volatility_annual < 20:  # Slightly more generous threshold
                risk_level = "low"
            elif volatility_annual < 35:  # Slightly more generous threshold
                risk_level = "medium"
            else:
                risk_level = "high"
            
            # ------- RATING CALCULATION -------
            # 1. Historical return (1-year): 30% weight (increased from 20%)
            one_year_return = (close_prices[-1] / close_prices[-min(252, len(close_prices)-1)] - 1) * 100
            
            # 2. Recent momentum (6-month): 25% weight (increased from 15%)
            six_month_return = (close_prices[-1] / close_prices[-min(126, len(close_prices)-1)] - 1) * 100
            
            # 3. Trend stability (3-month): 15% weight (unchanged)
            # Calculate R-squared of a linear fit to the last 3 months of prices
            recent_prices = close_prices[-min(63, len(close_prices)):]
            x = np.arange(len(recent_prices))
            slope, intercept, r_value, p_value, std_err = stats.linregress(x, recent_prices)
            trend_stability = r_value ** 2
            # If trend is positive, we can consider higher trend stability as better
            trend_direction = 1 if slope > 0 else -1
            
            # 4. Risk-adjusted return (Sharpe Ratio): 20% weight (reduced from 30%)
            # Assuming risk-free rate of 2%
            risk_free_rate = 0.02
            excess_return = (one_year_return / 100) - risk_free_rate
            sharpe_ratio = (excess_return / (volatility_annual / 100)) if volatility_annual > 0 else 0
            
            # 5. Predicted future return (next 60 days): 10% weight (reduced from 20%)
            # Use the existing model to predict future prices if available
            future_return = 0
            if model is not None and scaler_diff is not None:
                try:
                    # Prepare for prediction
                    close_prices_array = hist_data['Close'].values.astype(float).reshape(-1, 1)
                    diff_close_prices = np.diff(close_prices_array, axis=0)
                    diff_scaled = scaler_diff.transform(diff_close_prices)
                    
                    # Get last sequence
                    last_sequence = torch.tensor(diff_scaled[-seq_length:].reshape(1, seq_length, 1), dtype=torch.float32).to(device)
                    
                    # Predict future prices
                    last_price = close_prices_array[-1][0]
                    future_prices = [last_price]
                    
                    with torch.no_grad():
                        current_sequence = last_sequence
                        
                        for _ in range(prediction_days):
                            # Predict next difference
                            pred_diff_scaled = model(current_sequence)
                            pred_diff = scaler_diff.inverse_transform(pred_diff_scaled.cpu().numpy())[0][0]
                            
                            # Calculate next price
                            next_price = future_prices[-1] + pred_diff
                            future_prices.append(next_price)
                            
                            # Update sequence
                            current_seq_np = current_sequence.cpu().numpy()
                            current_seq_np = current_seq_np[:, 1:, :]
                            pred_for_seq = pred_diff_scaled.cpu().numpy().reshape(1, 1, 1)
                            new_sequence = np.concatenate([current_seq_np, pred_for_seq], axis=1)
                            current_sequence = torch.tensor(new_sequence, dtype=torch.float32).to(device)
                    
                    # Calculate predicted return
                    future_return = (future_prices[-1] / future_prices[0] - 1) * 100
                except Exception as e:
                    print(f"Prediction failed for {ticker}: {e}")
                    # If model prediction fails, calculate momentum and extrapolate
                    recent_slope = (close_prices[-1] - close_prices[-22]) / close_prices[-22] * 100  # ~1 month trend
                    future_return = recent_slope * 2  # Simple extrapolation based on recent momentum
            else:
                # If model isn't available, use recent momentum as a proxy
                recent_weeks = min(22, len(close_prices)-1)  # ~1 month of trading days
                future_return = (close_prices[-1] / close_prices[-recent_weeks] - 1) * 100 * 2  # Extrapolated 2 months
            
            # ------- ENHANCED NORMALIZATION -------
            # More generous normalization to better reward exceptional performance
            
            # 1. One-year return (higher range to properly score exceptional stocks like NVDA)
            # Old: normalized_one_year = min(100, max(0, one_year_return + 20))  # -20% to 80% → 0 to 100
            # New: Allow for recognizing returns up to 200%
            normalized_one_year = min(100, max(0, (one_year_return / 2) + 50))  # -100% to 150% → 0 to 100
            
            # 2. Six-month return (higher range)
            # Old: normalized_six_month = min(100, max(0, six_month_return * 2 + 20))  # -10% to 40% → 0 to 100
            # New: Allow for recognizing higher returns
            normalized_six_month = min(100, max(0, six_month_return + 50))  # -50% to 50% → 0 to 100
            
            # 3. Trend stability - now consider direction
            # Upward trend is better than downward trend with same stability
            normalized_trend = trend_stability * 100 * (1.5 if trend_direction > 0 else 0.5)  # 0 to 150 for positive, 0 to 50 for negative
            normalized_trend = min(100, max(0, normalized_trend))  # Cap at 100
            
            # 4. Sharpe ratio (more generous scaling)
            # Old: normalized_sharpe = min(100, max(0, sharpe_ratio * 25 + 50))  # -2 to 2 → 0 to 100
            # New: Better recognize high Sharpe ratios
            normalized_sharpe = min(100, max(0, sharpe_ratio * 15 + 40))  # -2.67 to 4 → 0 to 100
            
            # 5. Future return (more generous)
            # Old: normalized_future = min(100, max(0, future_return * 5 + 20))  # -4% to 16% → 0 to 100
            # New: Allow for higher projected returns
            normalized_future = min(100, max(0, future_return * 3 + 40))  # -13.33% to 20% → 0 to 100
            
            # ------- ADJUSTED WEIGHTS -------
            # Apply new weights (increased historical components, reduced future prediction)
            # volatility_penalty = min(metrics['volatility_annual'] / 1000, 1.0)
            # trend_penalty = -0.05 if metrics['trend_direction'] == 'negative' else 0
            
            weighted_score = (
                normalized_one_year * 0.30 +
                normalized_six_month * 0.25 +
                normalized_trend * 0.10 +
                normalized_sharpe * 0.30 +
                normalized_future * 0.02  # drastically reduced        # small penalty for downtrend
            )
            
            # ------- IMPROVED RATING SCALE -------
            # Adjust thresholds to be more reasonable
            if weighted_score >= 40:      # Was 80
                rating = 5
            elif weighted_score >= 30:    # Was 60
                rating = 4
            elif weighted_score >= 20:    # Was 40
                rating = 3
            elif weighted_score >= 10:    # Was 20
                rating = 2
            else:
                rating = 1
            
            # Add company name and sector information if available
            company_info = {}
            try:
                info = ticker_obj.info
                company_info = {
                    "name": info.get("shortName", ""),
                    "sector": info.get("sector", ""),
                    "industry": info.get("industry", "")
                }
            except:
                pass  # Skip if info retrieval fails
            
            # Store the results
            results[ticker] = {
                "risk_level": risk_level,
                "rating": rating,
                "company_info": company_info,
                "metrics": {
                    "volatility_annual": round(volatility_annual, 2),
                    "one_year_return": round(one_year_return, 2),
                    "six_month_return": round(six_month_return, 2),
                    "trend_stability": round(trend_stability, 2),
                    "trend_direction": "positive" if trend_direction > 0 else "negative",
                    "sharpe_ratio": round(sharpe_ratio, 2),
                    "predicted_future_return": round(future_return, 2),
                    "weighted_score": round(weighted_score, 2)
                }
            }
            
        except Exception as e:
            results[ticker] = {"error": f"Failed to analyze {ticker}: {str(e)}"}
    
    return results

# BiLSTM Model definition (same as in previous code)
class BiLSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=1):
        super(BiLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers * 2, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers * 2, batch_size, self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

'''
We need this function to lower ratings for risky stocks with extremely poor performance indicators.
How we did this:
We checked for high-risk stocks with very high volatility, negative trends, poor returns, and low stability, then reduced their ratings.

How we got to know:
We noticed some stocks (like IDEA) had unrealistic metrics and didn’t match their high rating, signaling the need for adjustment.
'''

def adjust_ratings(data):
    for key, val in data.items():
        if 'metrics' not in val or 'risk_level' not in val:
            continue
        metrics = val['metrics']
        if (
            val['risk_level'] == 'high' and
            metrics.get('volatility_annual', 0) > 1000 and
            metrics.get('sharpe_ratio', 1) <= 0 and
            metrics.get('trend_direction') == 'negative' and
            metrics.get('trend_stability', 1) < 0.3 and
            metrics.get('one_year_return', 0) < 0
        ):
            val['rating'] = min(val['rating'], 2)
    return data

# usage  : data = adjust_ratings(get_stock_ratings(['AAPL','MSFT','NVDA','GOOG','IDEA'])) -> gives dict


In [229]:
# dat = adjust_ratings(get_stock_ratings(['AAPL','MSFT','NVDA','GOOG','IDEA','META','AMZN']))